<a href="https://colab.research.google.com/github/ArnyWu/-Preview-AI-/blob/main/week12_%E5%BB%BA%E6%A7%8B%E6%8A%95%E8%B3%87%E7%B5%84%E5%90%88_with_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U google-generativeai>=0.8.3

In [ ]:
import google.generativeai as genai

In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
!pip install google-api-python-client

In [ ]:
!pip install beautifulsoup4
!pip install google

In [50]:
import yfinance as yf
import datetime as dt
import pandas as pd
import google.generativeai as genai
import requests
from bs4 import BeautifulSoup

In [66]:
# 下載股市資料並儲存為 CSV
def download_stock_data(stock_id):
    start = dt.datetime(2023, 11, 11)
    end = dt.datetime(2024, 11, 27)

    # 下載股市歷史資料
    stock_data = yf.download(stock_id, start=start, end=end).reset_index()
    stock_data["Date"] = pd.to_datetime(stock_data["Date"])
    stock_data.set_index("Date", inplace=True)
    stock_data.columns = stock_data.columns.droplevel('Ticker')

    # 取得各種財務資料
    stock = yf.Ticker(stock_id)
    company_info = stock.info  # 公司資料
    financials = stock.financials  # 損益表
    balance_sheet = stock.balance_sheet  # 資產負債表
    cashflow = stock.cashflow  # 現金流量表
    history = stock.history(period="1y")  # 取得歷史資料，包括股利與股票分割

    # 組合資料
    combined_data = {
        "Company Info": company_info,
        "Financials": financials,
        "Balance Sheet": balance_sheet,
        "Cashflow": cashflow,
        "Stock Data": stock_data,
        "History": history
    }

    # 儲存為 CSV 文件
    csv_filename = f"{stock_id}_stock_data.csv"
    stock_data.to_csv(csv_filename)
    print(f"Stock data saved to {csv_filename}")

    return csv_filename, combined_data

# 上傳檔案並請求模型進行分析
def analyze_stock_with_file(csv_filename):
    # 上傳 CSV 文件
    uploaded_file = genai.upload_file(csv_filename)

    # 請求模型生成分析內容
    response = model.generate_content(["請用zh-tw分析以下股票資，提供專業的股票分析報告。以及投資建議與最佳投資標的選擇。並提供其可看性評分(1-10分)：", uploaded_file])
    return response.text

In [67]:
# 設定股票代號
stock_id = "3231.TW"  # 用戶輸入股票代號

# 下載股市資料並儲存為 CSV
csv_filename, combined_data = download_stock_data(stock_id)

# 顯示抓取的資料
print(f"抓取的公司資訊：\n{combined_data['Company Info']}")
print(f"抓取的財務資料：\n{combined_data['Financials']}")
print(f"抓取的資產負債表：\n{combined_data['Balance Sheet']}")
print(f"抓取的現金流量表：\n{combined_data['Cashflow']}")
print(f"抓取的股市資料：\n{combined_data['Stock Data'].head()}")
print(f"抓取的歷史資料：\n{combined_data['History'].head()}")

# 使用 CSV 檔案進行分析
analysis_result = analyze_stock_with_file(csv_filename)
print(f"股票分析結果:\n{analysis_result}")

[*********************100%***********************]  1 of 1 completed


Stock data saved to 3231.TW_stock_data.csv
抓取的公司資訊：
{'address1': 'No. 158, Singshan Road', 'address2': 'Neihu', 'city': 'Taipei', 'zip': '11469', 'country': 'Taiwan', 'phone': '886 2 6616 9999', 'fax': '886 2 6612 5188', 'website': 'https://www.wistron.com', 'industry': 'Computer Hardware', 'industryKey': 'computer-hardware', 'industryDisp': 'Computer Hardware', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Wistron Corporation, together with its subsidiaries, designs, manufactures, and sells information technology products in Taiwan, Asia, and internationally. The company engages in the research, development, design, manufacture, testing, and sale of desktop computers, notebook computers, motherboards, servers, system platforms, high-speed and multi-function multiple-CPU computer systems, multi-media computers, network computers, consumer-type computers and special computers, microprocessors, CD-ROMs, PDAs, panel PCs, pocket comp